# 1. Super用法

首先明确，self代表了类的一个实例

super的作用一般包括以下两点：
- 用来在子类中调用父类的方法
- 多用于多继承问题中，解决查找顺序（MRO）、重复调用（钻石继承）等种种问题


## 1. 1 在子类中调用父类

B是A的子类，我们在B中可以直接调用A的方法。A.funxx(self)中self指代B的实例对象b，表示实例对象b通过A类名调用方法funxx()。

In [99]:
class A:
    def funxx(self):
        print(f'输入A的funxx方法的实例: {self}')
        print("执行 A 中的 funxx 方法 ... ...")
 
        
class B(A):
    def funxx(self):
        print(f'B的实例b: {self}')
        A.funxx(self)       # 通过类名调用父类中的同名方法，self 参数代表 B 类的实例对象 b
        print("执行 B 中的 funxx 方法 ... ...")

        
b = B()
print(f'我们定义的实例b: {b}')
b.funxx()


我们定义的实例b: <__main__.B object at 0x7fb948465a30>
B的实例b: <__main__.B object at 0x7fb948465a30>
输入A的funxx方法的实例: <__main__.B object at 0x7fb948465a30>
执行 A 中的 funxx 方法 ... ...
执行 B 中的 funxx 方法 ... ...


- 可以看出，三个实例内存地址相同，说明为同一个对象。

本质上，实例方法就是第一个参数为实例的方法。我们可以先把实例方法拿出来，当作一个不同的输入一个实例的函数；或者通过特殊的"实例x.方法()"的形式指定输入方法的第一个参数为实例x。
- 有如下两种写法：

In [217]:
# 先拿出A.funxx()，这是一个接受参数为一个实例的函数，我们输入实例A()
A.funxx(A())

输入A的funxx方法的实例: <__main__.A object at 0x7fb9500eda90>
执行 A 中的 funxx 方法 ... ...


In [219]:
# 这是通常的写法，直接指定funxx()接受的第一个参数为实例A()
A().funxx()

输入A的funxx方法的实例: <__main__.A object at 0x7fb9500ed700>
执行 A 中的 funxx 方法 ... ...


我们可以通过super函数调用。super引用父类而不必显式地指定它们的名称，从而令代码更易维护。

super具有两个参数，其中：
- 第一个参数则给出搜索目标方法的范围。
- 第二个参数给出 MRO（方法解析顺序），也就是搜索目标方法的顺序。

具体来看：
- 第一个参数为一个类，指定了从这个类的下一个父类（MRO链条上最近的父类）开始找。
- 第二个参数为一个类或者实例，如果是类，这个类所在的MRO链条将成为搜索目标方法的顺序；如果是实例，这个实例所在的MRO链条将成为搜索目标方法的顺序。

- 第二个参数一般为self，代表了从调用super方法的这个类的实例。这个实例所在的类（也就是调用super方法的类）所在的MRO链条将成为搜索目标方法的顺序。

super的第二个参数（super(type, obj): obj must be an instance or subtype of type）：
- 如果是实例，则一定要是super的第一个参数的类或者其子类的实例
- 如果是类，则一定要是调用super的类本身或者子类

- 可以这样理解：如果super的第二个参数是类，那么super返回的也是一个类；第二个参数是实例，则super返回的也是一个实例

super方法沿着第二个参数指定的MRO链条，从第一个参数的下一个父类开始搜索节点，返回离的最近的节点（可能是实例也可能是类）

因此，我们使用super方法改写上面的代码：

In [204]:
class A:
    def funxx(self):
        print(f'输入A的funxx方法的实例: {self}')
        print("执行 A 中的 funxx 方法 ... ...")
 
        
class B(A):
    def funxx(self):
        print(f'B的实例b: {self}')
        super(B, self).funxx()       # 通过类名调用父类中的同名方法，self 参数代表 B 类的实例对象 b
        print("执行 B 中的 funxx 方法 ... ...")

        
b = B()
print(f'我们定义的实例b: {b}')
b.funxx()

我们定义的实例b: <__main__.B object at 0x7fb9500ed700>
B的实例b: <__main__.B object at 0x7fb9500ed700>
输入A的funxx方法的实例: <__main__.B object at 0x7fb9500ed700>
执行 A 中的 funxx 方法 ... ...
执行 B 中的 funxx 方法 ... ...


这里super(B, self)中的参数说明：
- 第一个参数B表示从B的下一个节点开始搜索，即从离B的最近的父类开始搜索，一直向上搜索到根节点。
- 第二个参数self代表了实例b，表示其所在类的继承顺序（MRO）为：B→A→object。所以调用时是在 B 的父类 A 中寻找，如找不到目标方法则会在更上一层的object中寻找。注意这里我们通过实例b给出继承顺序

还可以写成如下等价形式。super(B, B)返回的是第二个参数

In [205]:
class A:
    def funxx(self):
        print(f'输入A的funxx方法的实例: {self}')
        print("执行 A 中的 funxx 方法 ... ...")
 
        
class B(A):
    def funxx(self):
        print(f'B的实例b: {self}')
        super(B, B).funxx(self)       # 通过类名调用父类中的同名方法，self 参数代表 B 类的实例对象 b
        print("执行 B 中的 funxx 方法 ... ...")

        
b = B()
print(f'我们定义的实例b: {b}')
b.funxx()


我们定义的实例b: <__main__.B object at 0x7fb9500ed910>
B的实例b: <__main__.B object at 0x7fb9500ed910>
输入A的funxx方法的实例: <__main__.B object at 0x7fb9500ed910>
执行 A 中的 funxx 方法 ... ...
执行 B 中的 funxx 方法 ... ...


如果不指定super的第一个参数，则第一个参数默认为当前调用super的类(__class__)；不指定第二个参数则第二个参数默认为self。因此，上述代码等价于：

In [55]:
class A:
    def funxx(self):
        print("执行 A 中的 funxx 方法 ... ...")
 
        
class B(A):
    def funxx(self):
        print(__class__)
        super().funxx()       # 通过类名调用父类中的同名方法，self 参数代表 B 类的实例对象 b
        print("执行 B 中的 funxx 方法 ... ...")

        
b = B()
b.funxx()


<class '__main__.B'>
执行 A 中的 funxx 方法 ... ...
执行 B 中的 funxx 方法 ... ...


再看一个例子：

In [112]:
class A:
    def funxx(self):
        print("找到 funxx() 位于 A 中...")
 
class B(A):
    pass
 
 
class C(A):
    def funxx(self):
        print("找到 funxx() 位于 C 中...")
 
 
class D(A):
    pass
 
 
class E(B, C):
    pass
 
 
class F(E, D):
    def funff(self):
        print("执行 F 中的 funff()...")
        super(E, self).funxx()
 
        
print(f"F 类的 MRO : {F.__mro__}")
f = F()
f.funff()

F 类的 MRO : (<class '__main__.F'>, <class '__main__.E'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.D'>, <class '__main__.A'>, <class 'object'>)
执行 F 中的 funff()...
找到 funxx() 位于 C 中...


在这个例子中，F 类的 MRO：F→E→B→C→D→A→object，super(E, self)表示从E类的下一个节点开始找，继承顺序由self指定，代表f所在的类F的MRO链条为搜索链条。先在B中寻找，没有找到，然后在C中寻找，找到了，并且运行C().funxx。

如果我们改变super的第一个参数为D，那么就会从D开始寻找：

In [142]:
class A:
    def funxx(self):
        print("找到 funxx() 位于 A 中...")
 
class B(A):
    pass
 
 
class C(A):
    def funxx(self):
        print("找到 funxx() 位于 C 中...")
 
 
class D(A):
    pass
 
 
class E(B, C):
    pass
 
 
class F(E, D):
    def funff(self):
        print("执行 F 中的 funff()...")
        super(D, self).funxx()
 
        
print(f"F 类的 MRO : {F.__mro__}")
f = F()
f.funff()

F 类的 MRO : (<class '__main__.F'>, <class '__main__.E'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.D'>, <class '__main__.A'>, <class 'object'>)
执行 F 中的 funff()...
找到 funxx() 位于 A 中...


注意，super的第一个参数类最好是调用super的类本身或者父类。否则从子类开始找，会找到调用super的类的方法，然后又会进入super函数，出现无限递归并且溢出的情况，除非使用super的方法和super调用的父方法名称不同。

In [240]:
class A:
    def funxx(self):
        print("找到 funxx() 位于 A 中...")
 
class B(A):
    pass
 
 
class C(A):
    def funxx(self):
        print("找到 funxx() 位于 C 中...")
 
 
class D(A):
    pass
 
 
class E(B, C):
    def funxx(self):
        super(F, F()).funxx()
 
 
class F(E, D):
    def funff(self):
        print("执行 F 中的 funff()...")
        super(F, self).funxx()

        
print(f"F 类的 MRO : {F.__mro__}")
e = E()
e.funxx()

F 类的 MRO : (<class '__main__.F'>, <class '__main__.E'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.D'>, <class '__main__.A'>, <class 'object'>)


RecursionError: maximum recursion depth exceeded while calling a Python object

此处出现了无限递归溢出的情况

super的第二个参数（super(type, obj): obj must be an instance or subtype of type）：
- 如果是实例，则一定要是super的第一个参数的类或者其子类的实例
- 如果是类，则一定要是调用super的类本身或者子类

In [243]:
class A:
    def funxx(self):
        print("找到 funxx() 位于 A 中...")
 
class B(A):
    pass
 
 
class C(A):
    def funxx(self):
        print("找到 funxx() 位于 C 中...")
 
 
class D(A):
    pass
 

class E(B, C):
    def funxx(self):
        print(super(E, F) == C)
        print(super(E, E))
        print(super(E, F).funxx)
        super(E, F()).funxx()
 
 
class F(E, D):
    def funff(self):
        print("执行 F 中的 funff()...")
        super(F, self).funxx()
 
        
print(f"F 类的 MRO : {F.__mro__}")
e = E()
e.funxx()

F 类的 MRO : (<class '__main__.F'>, <class '__main__.E'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.D'>, <class '__main__.A'>, <class 'object'>)
False
<super: <class 'E'>, <E object>>
<function C.funxx at 0x7fb948856e50>
找到 funxx() 位于 C 中...


## 1.2 重复调用问题

In [234]:
class A:
    def __init__(self):
        print("打印属性 a")
 
class B(A):
    def __init__(self):
        print("打印属性 b")
        A.__init__(self)
 
   
class C(A):
    def __init__(self):
        print("打印属性 c")
        A.__init__(self)

   
class D(B, C):
    def __init__(self):
        print("打印属性 d")
        B.__init__(self)
        C.__init__(self)
    
d = D()
print(D.__mro__)

打印属性 d
打印属性 b
打印属性 a
打印属性 c
打印属性 a
(<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>)


因为 B，C 都继承自 A，所以当 D 在实例化时，A 的构造函数被执行了两次。这就是所谓的重复调用问题。我们可以使用super函数解决。

In [302]:
class A:
    def __init__(self):
        print("打印属性 a", self)
 
   
class B(A):
    def __init__(self):
        print("打印属性 b", self)
        super().__init__()  # super() 等同于 super(B, self)
 
   
class C(A):
    def __init__(self):
        print("打印属性 c", self)
        super().__init__()  # super() 等同于 super(C, self)
 
 
class D(B, C):
    def __init__(self):
        print("打印属性 d", self)
        super(D, self).__init__()
 
   
d = D()

打印属性 d <__main__.D object at 0x7fb9485061f0>
打印属性 b <__main__.D object at 0x7fb9485061f0>
打印属性 c <__main__.D object at 0x7fb9485061f0>
打印属性 a <__main__.D object at 0x7fb9485061f0>


查看输出结果我们发现虽然解决了重复调用问题，但是输出结果的顺序好像与我们想的有所区别。我们的惯性思维是：先执行 D 类的__init__() 方法，接着调用 B 类的__init__() 方法，B 类的构造方法中又调用了父类 A 的__init__() 方法，然后再是调用 C 类的__init__() 方法，该方法也调用了父类A的__init__() 方法。所以执行的结果应该是：打印属性 d，打印属性 b，打印属性 a，打印属性 c。

为何结果不是我们想的那样呢，首先我们要知道 D 类中的第二个参数 self 为 D 的实例 d，它提供的 MRO 为：D→B→C→A→object。所以 D 类中的 super() 函数产生的是 d 的代理对象，当其调用父类 B 的__init__() 时，B 的 super() 的第二个参数为 D 中的 super object，其所提供的 MRO 依旧为：D→B→C→A→object。也就是说 B 中的 super() 调用的是它的上一级 C 中的__init__() ，而不是 A 中的__init__()。所以执行的结果是：打印属性 d，打印属性 b，打印属性 c，打印属性 a。

当然，也可以实现一个重复的效果：

In [304]:
class A:
    def __init__(self):
        print("打印属性 a", self)
 
   
class B(A):
    def __init__(self):
        print("打印属性 b", self)
        super(C, self).__init__()  # super() 等同于 super(B, self)
        super(B, self).__init__()
 
   
class C(A):
    def __init__(self):
        print("打印属性 c", self)
        super().__init__()  # super() 等同于 super(C, self)
 
 
class D(B, C):
    def __init__(self):
        print("打印属性 d", self)
        super(D, self).__init__()
 
   
d = D()

打印属性 d <__main__.D object at 0x7fb948524610>
打印属性 b <__main__.D object at 0x7fb948524610>
打印属性 a <__main__.D object at 0x7fb948524610>
打印属性 c <__main__.D object at 0x7fb948524610>
打印属性 a <__main__.D object at 0x7fb948524610>


## 1.3 类方法和静态方法中super的用法

和实例方法的用法完全相同。只需记住：
- super的第二个参数是实例，就可以理解为返回的是实例；第二个参数是类就可以理解为返回的是类。

例1：此处沿着类C的MRO链条，从距离B的最近的父类A开始搜索。

In [292]:
class A:
    @classmethod
    def hello(cls):
        print("Hello from A")
class B(A):
    @classmethod
    def hello(cls):
        print('Hello from B')
class C(B):
    @classmethod
    def hello(cls):
        super(B, B).hello()

In [293]:
C().hello()

Hello from A


例2：注意到类方法的第一个参数也可以传入实例（此时会把实例对应的类传递给类方法的第一个参数）：

In [273]:
class A:
    @classmethod
    def hello(cls):
        print("Hello from A")
class B(A):
    @classmethod
    def hello(cls):
        print('Hello from B')
class C(B):
    @classmethod
    def hello(cls):
        super(B, C()).hello()

In [274]:
C().hello()

Hello from A


静态方法同理

In [277]:
class A:
    @staticmethod
    def hello():
        print("Hello from A")
class B(A):
    @staticmethod
    def hello():
        print('Hello from B')
class C(B):
    @staticmethod
    def hello():
        super(B, C).hello()

In [278]:
C().hello()

Hello from A
